In [15]:
import random
import torch

# Load your data from the .pt file
file_path = "/gpfs/proj1/choe_lab/tanu/Species_classification/features_extracted/3_species/train/cricket_data_feature_extracted.pt"

data = torch.load(file_path)



In [16]:
print(len(data))
print(data[0])
print(type(data))

2273
{'array': array([[-1.2775939 , -1.2775939 , -1.2775939 , ..., -0.06692787,
        -0.11456262, -0.10012668],
       [-1.2775939 , -1.2775939 , -1.2775939 , ..., -0.07882945,
        -0.10893779, -0.2332645 ],
       [-1.2775939 , -1.2775939 , -1.2775939 , ..., -0.08546656,
        -0.13467082, -0.11205074],
       ...,
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237],
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237],
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237]], dtype=float32), 'label': 'Gryllus lineaticeps'}
<class 'list'>


In [17]:
# Set the desired number of instances for each class
desired_instances = 750

In [18]:

# Create a dictionary to organize the data by class label
def waveforms_wih_classes(data):
  class_data = {}

  # Organize the data into classes
  for instance in data:
      waveform = instance['array']
      label = instance['label']

      if label not in class_data:
          class_data[label] = []
      class_data[label].append(waveform)
  class_data = dict(sorted(class_data.items(), key = lambda item: item[0]))
  return class_data

In [19]:
class_data = waveforms_wih_classes(data)

In [20]:
print(class_data.keys())

dict_keys(['Gryllus armatus', 'Gryllus lightfooti', 'Gryllus lineaticeps'])


In [30]:
print(len(class_data['Gryllus lightfooti']))
print(data[0])

1212
{'array': array([[-1.2775939 , -1.2775939 , -1.2775939 , ..., -0.06692787,
        -0.11456262, -0.10012668],
       [-1.2775939 , -1.2775939 , -1.2775939 , ..., -0.07882945,
        -0.10893779, -0.2332645 ],
       [-1.2775939 , -1.2775939 , -1.2775939 , ..., -0.08546656,
        -0.13467082, -0.11205074],
       ...,
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237],
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237],
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237]], dtype=float32), 'label': 'Gryllus lineaticeps'}


In [22]:
#Appply only undersampling 
def undersampling_data(class_data):
    undersampled_data = []
    for label, waveforms in class_data.items():
        num_instances = len(waveforms)
        if num_instances > desired_instances:
            #undersample by randomly selecting instances to keep
            undersampled_waveforms = random.sample(waveforms, desired_instances)
            undersampled_data.extend([{'array': waveform, 'label': label} for waveform in undersampled_waveforms])
        else:
            undersampled_data.extend([{'array': waveform, 'label': label} for waveform in waveforms])

    return undersampled_data
    

In [23]:
#balance the 3 species set 750 (undersample last species)
balanced_3_species =  undersampling_data(class_data)
len(balanced_3_species)

1811

In [24]:
balanced_waveforms_per_class_3species = waveforms_wih_classes(balanced_3_species)
print(len(balanced_waveforms_per_class_3species["Gryllus lightfooti"]))

750


In [67]:
balanced_5_species =  undersampling_data(class_data)

In [68]:
len(balanced_5_species)

2527

In [71]:
balanced_waveforms_per_class_5species = waveforms_wih_classes(balanced_5_species)
print(len(balanced_waveforms_per_class_5species["Gryllus lightfooti"]))

650


In [1]:
# Apply oversampling and undersampling

def balancing_data(class_data):
  # Create new lists to store the balanced data
  oversampled_data = []
  undersampled_data = []
  for label, waveforms in class_data.items():
      # Determine the number of instances in the current class
      num_instances = len(waveforms)

      if num_instances < desired_instances:
          # Oversample by duplicating existing instances
          oversampled_waveforms = waveforms
          while len(oversampled_waveforms) < desired_instances:
              instance_to_add = random.choice(waveforms)
              oversampled_waveforms.append(instance_to_add)
          oversampled_data.extend([{'array': waveform, 'label': label} for waveform in oversampled_waveforms])
      elif num_instances > desired_instances:
          # Undersample by randomly selecting instances to keep
          undersampled_waveforms = random.sample(waveforms, desired_instances)
          undersampled_data.extend([{'array': waveform, 'label': label} for waveform in undersampled_waveforms])
      else:
          # No oversampling or undersampling needed
          oversampled_data.extend([{'array': waveform, 'label': label} for waveform in waveforms])
          undersampled_data.extend([{'array': waveform, 'label': label} for waveform in waveforms])

  return oversampled_data,undersampled_data


In [51]:
oversampled_data,undersampled_data = balancing_data(class_data)

In [52]:
balanced_data = (oversampled_data + undersampled_data)

In [53]:
print(len(balanced_data))
print(balanced_data[0])

4500
{'array': array([[-0.72662544, -1.2775939 , -1.0434303 , ...,  0.09354903,
        -0.1369554 , -0.13669997],
       [-0.9523918 , -1.2775939 , -1.2690147 , ..., -0.01750982,
        -0.18199639, -0.15710948],
       [-0.91816765, -1.2775939 , -1.2413666 , ..., -0.07037605,
        -0.14542983, -0.04364534],
       ...,
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237],
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237],
       [ 0.46703237,  0.46703237,  0.46703237, ...,  0.46703237,
         0.46703237,  0.46703237]], dtype=float32), 'label': 'Gryllus assimilis'}


In [54]:
balanced_waveforms_per_class = waveforms_wih_classes(balanced_data)

In [55]:
print(len(balanced_waveforms_per_class))
print(len(balanced_waveforms_per_class['Gryllus cohni']))

9
500


In [26]:
output_filepath = "/gpfs/proj1/choe_lab/tanu/Species_classification/features_extracted/3_species/balanced_train_750_2273/cricket_data_feature_extracted.pt"
# Save the balanced_training_data to the specified location
torch.save(balanced_3_species, output_filepath)